In [11]:
import numpy as np
from astropy.io import fits
from astropy.wcs import WCS
from reproject import reproject_interp
from astropy import wcs
import os
from spectral_cube import SpectralCube
import warnings
warnings.filterwarnings('ignore')

In [12]:
datadir='/path/to/fits/'
fits_file_org=datadir+'fits1.fits'
fits_file_proj=datadir+'fits2.fits'
fits_out=datadir+'fits2_jypbm_regrids.fits'

hdu1 = fits.open(fits_file_org)[0]
hdu2 = fits.open(fits_file_proj)[0]


In [ ]:
#WCS axes have to match for both files
#may need to rewrite fits_file.fits

#(1) cut dimension 1 axis
os.system('rm -rf '+fits_file_proj.strip('.fits')+'_squeeze.fits')
data=hdu2.data.squeeze()
header=hdu2.header
wmap=wcs.WCS(header)
new_wmap=wmap.dropaxis(2)

#(2)swap X and Y axes
new_wmap2=new_wmap.swapaxes(0,1)

#(3) rewrite wcs to header
new_header=new_wmap2.to_header()

#(4) make new fits file with all changes
newfits=fits.PrimaryHDU(data=data,header=new_header)
newfits.writeto(fits_file_proj.strip('.fits')+'_squeeze.fits')
fits2new=fits_file_proj.strip('.fits')+'_squeeze.fits'
hdu2 = fits.open(fits2new)[0]

#(optional) may need to fix header as well
#hdu2.header.remove('CRPIX3')
#hdu2.header.remove('CRVAL3')
#hdu2.header.remove('CDELT3')
#hdu2.header.remove('CUNIT3')
#hdu2.header.remove('CTYPE3')
#hdu2.header['WCSAXES']=2
#hdu1.header['BUNIT']='Jy/bm'



In [ ]:
#for 2D images
array, footprint = reproject_interp(hdu2, hdu1.header)
fits.writeto(fits_out, array, hdu1.header, clobber=True)

In [ ]:
#for 3D cubes
header = fits.getdata(fits_file_org, header=True)[1]
X=SpectralCube.read(fitsfile)
X.allow_huge_operations=True
X.reproject(header)
X.write(fits_out)